In [3]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as plt
import torch as pt
import csv
import pickle #to save notebook at sessions


#from Bojar lab format
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

#set path for pickles to be saved in
pickle_path = '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/python pickles/'

# Machine Learning Workflow

1. Random Forest Classifier on all T-cells (just glyco vs glyco+genes identified in 1
- Random Forest on all T-cells w normalized data
2. Random Forest Classifier on T-cell subsets
3. Identify genes whose expression levels correlate with glycogene expression levels (some correlation algorithm), repeat 1 & 2 with enhanced glycogene set


## 1. Random Forest Clustering using all T-cells and raw data
to build model(S) --> build on all T-cells, and then by T-cell subtypes
https://github.com/BojarLab/scGlycomics_b16_branching/blob/main/Random%20Forest%20-%20Apr%208%202022%20-%20RQ.ipynb

In [35]:
'''Load glycosorted RAW dataframe from pickle, saved from glycogene filtering raw.ipynb
Dataframes have:
- gene names as columns (243 glycogenes) + columns for type, biotin count, PHA-L score =246 columns total
- cell barcodes as row index, for cells identified as TILs via ProjecTILs package in R
'''
pickle_in = open(pickle_path + "glycosorted_TIL.pkl","rb")
glycosorted_TIL = pickle.load(pickle_in)

pickle_in = open(pickle_path + "glycosorted_LN.pkl","rb")
glycosorted_LN = pickle.load(pickle_in)

In [37]:
#make copies of dataframes to not accidentally change original df
glycoTIL = glycosorted_TIL.copy()
glycoLN = glycosorted_LN.copy()

### Split dataset into training, validation, and test set

In [39]:
'''
Generate training, validation, and test set from original full df using scikit learn 
TIL ver.
'''
#y: PHA-L score array
y = glycoTIL['PHA-L'].values 

#X: glycogene transcript data array
x = glycoTIL.iloc[:, :-3].values

# Split training, validation and test set
x_train_val, x_test, y_train_val, y_test = train_test_split(
    x, y, test_size=0.1, random_state=342, stratify=y)

x_train, x_val, y_train, y_val = train_test_split(
    x_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val)

### Random Forest classifier model training (all from Bojar lab)

In [38]:
# Parameters for grid search

# Number of trees in random forest
n_estimators = [int(x) for x in np.arange(200, 800, step=100)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.arange(10, 50, step=10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               }

In [40]:
''''Use RandomSearchCV to optimize hyperparameters'''
#takes a while to run!!
model = RandomForestClassifier()

model_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, 
                                  n_iter = 20, cv = 5, verbose=5, random_state=42, n_jobs = -1)

model_random.fit(x_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


KeyboardInterrupt: 

In [ ]:
# Return the best estimator
TILmodel_all_raw = model_random.best_estimator_
TILmodel_all_raw.get_params()

In [ ]:
# save pickle 
with open(pickle_path + 'TILmodel_all_raw.pkl', 'wb') as f:
    pickle.dump(TILmodel_all_raw, f)

f.close()

# open via: 
# #load updated df from pickle
# pickle_in = open("pickle_path + TILmodel_all_raw.pkl","rb")
# TILmodel_all_raw = pickle.load(pickle_in)

In [ ]:
def model_evaluation(model, x, y):
    print(f"Accuracy for 'PHA-L high' class: {100*(model.score(x[y==1], y[y==1])):>4f}%")
    print(f"Accuracy for 'PHA-L low' class: {100*(model.score(x[y==0], y[y==0])):>4f}%")
    print(f"Overall accuracy: {100*(model.score(x, y)):>4f}%")

    model_predict = model.predict(x)
    model_predict_prob = model.predict_proba(x)

    print(f"Average loss: {log_loss(y, model_predict_prob):>4f}")
    print(f"ROC Curve AUC: {roc_auc_score(y, model_predict):>4f}")
    print(f"F1 score: {f1_score(y, model_predict):>4f}")

In [ ]:
#training set
model_evaluation(TILmodel_all_raw, x_train, y_train)

In [ ]:
#validation 
model_evaluation(TILmodel_all_raw, x_val, y_val)

## 2B. Random Forest Classifier with LN/TIL combined T-cells
Train models with identified subtypes
https://github.com/BojarLab/scGlycomics_b16_branching/blob/main/Model%20training%20-%20Apr%2010%202022%20-%20RQ.ipynb


In [16]:
'''Load glycosorted RAW dataframe from pickle, saved from glycogene filtering raw.ipynb
Dataframes have:
- gene names as columns (243 glycogenes) + columns for type, biotin count, PHA-L score =246 columns total
- cell barcodes as row index, for cells identified as TILs via ProjecTILs package in R
'''
pickle_in = open(pickle_path + "glycosorted_TIL.pkl","rb")
glycosorted_TIL = pickle.load(pickle_in)

pickle_in = open(pickle_path + "glycosorted_LN.pkl","rb")
glycosorted_LN = pickle.load(pickle_in)

In [33]:
#make copies of dataframes to not accidentally change original df
glycoTIL = glycosorted_TIL.copy()
glycoLN = glycosorted_LN.copy()

In [26]:
glycoLN.shape

(316, 10595)

In [27]:
glycoTIL.shape

(316, 9824)

In [34]:
combo = pd.concat([glycoTIL, glycoLN])

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [29]:
combo.

,TACAGTGGTTCAACCA-1,ACATACGAGTGCCATT-1,TGTCCCATCGGCTTGG-1,ACTGTCCAGGATGGTC-1,GAACATCTCTGCTGTC-1,CATCAAGAGACAATAC-1,GAAACTCCAGATTGCT-1,CAGCATACAGCCTATA-1,TAGAGCTAGGAGTCTG-1,GGGATGAGTCAGGACA-1,...,GGCGTGTAGAAGGGTA-1,CGGAGTCAGCGCTTAT-1,GGACAAGAGTGGTAAT-1,CTGATAGAGAATGTGT-1,AGCGTATGTTCGAATC-1,AGATTGCCACGAAGCA-1,GGCGTGTAGGGATCTG-1,CGTCTACTCAGCCTAA-1,GCTGGGTCACCGGAAA-1,CTGAAGTGTCTCATCC-1
Ahsa1,0,0,0,0,3,2,2,0,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Api5,1,2,0,1,4,2,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Atp6v1e1,1,0,0,0,2,1,0,0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bcap31,1,0,1,1,7,2,1,1,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cops6,0,0,0,0,3,5,2,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Dsel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
Glce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,CD8_NaiveLike,CD8_NaiveLike,CD8_NaiveLike,CD8_NaiveLike,CD8_NaiveLike,CD8_NaiveLike,CD8_NaiveLike,CD8_NaiveLike,CD8_NaiveLike,CD8_NaiveLike
Biotin_hash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20,19,18,18,16,16,14,10,10,6


In [ ]:
'''
Generate training, validation, and test set from original full df using scikit learn 
TIL ver.
'''
#y: PHA-L score array
y = glycoTIL['PHA-L'].values 

#X: glycogene transcript data array
x = glycoTIL.iloc[:, :-3].values

# Split training, validation and test set
x_train_val, x_test, y_train_val, y_test = train_test_split(
    x, y, test_size=0.1, random_state=342, stratify=y)

x_train, x_val, y_train, y_val = train_test_split(
    x_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val)

In [ ]:
# Parameters for grid search

# Number of trees in random forest
n_estimators = [int(x) for x in np.arange(200, 800, step=100)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.arange(10, 50, step=10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               }

In [ ]:
''''Use RandomSearchCV to optimize hyperparameters'''
#takes a while to run!!
model = RandomForestClassifier()

combomodel_all_raw = RandomizedSearchCV(estimator = model, param_distributions = random_grid, 
                                  n_iter = 20, cv = 5, verbose=5, random_state=42, n_jobs = -1)

combomodel_all_raw.fit(x_train, y_train)

In [ ]:
# save pickle 
with open(pickle_path + 'combomodel_all_raw.pkl', 'wb') as f:
    pickle.dump(combomodel_all_raw, f)

f.close()

# open via: 
# #load updated df from pickle
# pickle_in = open("pickle_path + combomodel_all_raw.pkl","rb")
# combomodel_all_raw = pickle.load(pickle_in)

In [ ]:
# Return the best estimator
combomodel_all_raw = model_random.best_estimator_
combomodel_all_raw.get_params()

In [ ]:
#training set
model_evaluation(TILmodel_all_raw, x_train, y_train)

In [ ]:
#validation 
model_evaluation(TILmodel_all_raw, x_val, y_val)

## 3. Random Forest Classifier with T-cell subsets
Train models with identified subtypes
https://github.com/BojarLab/scGlycomics_b16_branching/blob/main/Model%20training%20-%20Apr%2010%202022%20-%20RQ.ipynb


In [7]:
##load updated df from pickle
pickle_in = open(pickle_path + "TILtcell_dfs.pkl","rb")
TILtcell_dfs = pickle.load(pickle_in)

pickle_in = open(pickle_path + "LNtcell_dfs.pkl","rb")
LNtcell_dfs = pickle.load(pickle_in)

In [21]:
#extracting all dataframes from df dictionary
CD8_NaiveLike_df= TILtcell_dfs['CD8_NaiveLike_df']
CD8_EffectorMemory_df = TILtcell_dfs['CD8_EffectorMemory_df']
Th1_df = TILtcell_dfs['Th1_df']
CD8_EarlyActiv_df = TILtcell_dfs['CD8_EarlyActiv_df']
Treg_df = TILtcell_dfs['Treg_df']
CD8_Tex_df = TILtcell_dfs['CD8_Tex_df']
CD4_NaiveLike_df = TILtcell_dfs['CD4_NaiveLike_df']
Tfh_df = TILtcell_dfs['Tfh_df']
CD8_Tpex_df = TILtcell_dfs['CD8_Tpex_df']